### Import packages

In [ ]:
import os
import sys
sys.path.append(r'{}'.format(os.path.abspath(os.pardir)))
# Custom packages
from wholebrain_tools import dataIO, aba

# Instantiate an Atlas object from the aba module
# The first time you run this it will download the structures.json file from the Allen Institute server
paths = dataIO.pathParser()
structuresFile = paths.structures
A = aba.Atlas(nodes=structuresFile)
DFM = aba.AnatomyDataFrameManager(A)


## Load data

In [ ]:
# ------------------------------------------------------------------------------
searchpath =  paths.alldata
# ------------------------------------------------------------------------------

# Load dots data from all aniamls
df = dataIO.allMiceDots(searchpath)
df = DFM.multiIndexDf_from_dotsDf(df)

# Dataframe at mid ontology
mid_df = DFM.dotsRegionsDf_to_mid(df, numberOfCells=True)

## Process and save

In [ ]:
# Perform the Leave-One-Out analysis at mid-ontology 
mid_df = DFM.dots_leaveOneOut_correction(
    mid_df,
    verbose=False,
    min_num_pnn=2,
    min_num_pv=2,
    min_num_mice=3,
    )

# Drop the dots in the fiber tracts
mid_df = mid_df.drop(1009, level='coarse')
# Average across Experimental units
mid_df = mid_df.groupby('params', axis=1).mean()
# Add the Acronym as a column
mid_df['acronym'] = A.ids_to_acronyms(mid_df.index.get_level_values('mid'))
mid_df = mid_df.droplevel('coarse')

mid_df.to_csv("dataFrameForBrainRender.csv")

print('Exported.')